In [1]:
import torch
from huggingface_hub import notebook_login
from transformers.utils import send_example_telemetry
from datasets import load_dataset, load_metric,ClassLabel, Sequence
import random
import transformers
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from IPython.display import display, HTML
from transformers import (AutoTokenizer, TFAutoModelForTokenClassification, create_optimizer, DataCollatorForTokenClassification,
                          pipeline,TFTrainer, TFTrainingArguments, TFRobertaForTokenClassification, RobertaForTokenClassification)
import nltk
from torch.optim import AdamW

In [2]:
#data= nltk.corpus.treebank.tagged_sents()
from sklearn.model_selection import train_test_split


data= nltk.corpus.treebank.tagged_sents()
training_data, testing_data = train_test_split(data, test_size=0.2, random_state=25)
x_train = list(map(lambda x: [i[0] for i in x],training_data))
x_test = list(map(lambda x: [i[0] for i in x],testing_data))
y= list(map(lambda x: [i[1] for i in x],data))
label_list = set([item for sublist in y for item in sublist])
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}
y_test = list(map(lambda x: [label2id[i[1]] for i in x],testing_data))
y_train = list(map(lambda x: [label2id[i[1]] for i in x],training_data))
print(label_list,len(label_list))

{'IN', 'JJ', 'JJS', 'SYM', '#', 'PRP', 'LS', 'WRB', ':', 'VBP', '``', 'PDT', 'CC', 'RBS', 'UH', '-RRB-', 'VBN', 'WDT', '-LRB-', 'PRP$', 'JJR', 'VBD', 'EX', 'VBG', 'RBR', "''", '$', 'MD', 'FW', 'RP', 'TO', 'CD', 'WP', 'DT', 'RB', 'NNS', '.', 'NNPS', 'POS', 'VB', '-NONE-', 'NNP', 'WP$', ',', 'NN', 'VBZ'} 46


In [3]:
print(x_train[2])
print(y_train[2])
i=10
print(len(x_train[i]),len(y_train[i]))

['Why', 'does', 'this', 'large', 'hypothetical', 'seller', 'trade', 'in', 'Chicago', 'instead', 'of', 'New', 'York', '*T*-1', '?']
[7, 45, 33, 1, 1, 44, 39, 0, 41, 34, 0, 41, 41, 40, 36]
46 46


In [4]:
class POS_dataset(Dataset):
    def __init__(self,x,y, tokenizer):
        self.tokenizer = tokenizer
        self.sentences = x
        self.labels = y
        self.pos_labels = self.get_pos_labels()
        self.encoded_inputs = self.tokenize_dataset()
        
        
    def tokenize_dataset(self):
        tokenized_inputs = tokenizer(self.sentences, padding=True,truncation = True, is_split_into_words=True)
        labels = []
        for i, label in enumerate(self.labels):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                # Special tokens have a word id that is None. We set the label to -100 so they are automatically
                # ignored in the loss function.
                if word_idx is None:
                    label_ids.append(-100)
                # We set the label for the first token of each word.
                elif word_idx != previous_word_idx:
                    label_ids.append(label[word_idx])
                # For the other tokens in a word, we set the label to either the current label or -100, depending on
                # the label_all_tokens flag.
                else:
                    label_ids.append(label[word_idx])
                previous_word_idx = word_idx

            labels.append(label_ids)

        tokenized_inputs["label_ids"] = labels
        return tokenized_inputs

    def get_pos_labels(self):
        unique_labels = set()
        for labels in self.labels:
            unique_labels.update(labels)
        return list(unique_labels)

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        
        
        
        
        return {
            "input_ids": self.encoded_inputs["input_ids"][idx],
            "attention_mask": self.encoded_inputs["attention_mask"][idx],
            "label_ids": self.encoded_inputs["label_ids"][idx],
            #'test': self.tokenizer.convert_ids_to_tokens(self.encoded_inputs["input_ids"][idx])
        }

        
        




task = "pos"  
model_checkpoint = "roberta-base"   #xlm-roberta-large-finetuned-conll03-english, 
batch_size = 16
model = RobertaForTokenClassification.from_pretrained(model_checkpoint,num_labels=len(label_list), 
                                                      id2label=id2label, label2id=label2id)
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer = transformers.RobertaTokenizerFast.from_pretrained(model_checkpoint,add_prefix_space=True)
test_ds = POS_dataset(x_train,y_train,tokenizer)
train_ds = POS_dataset(x_test,y_test,tokenizer)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

In [5]:
print(train_ds[2])
#print(train_ds[2]['input_ids'])
#print(train_ds[2]['test'])
#print(train_ds[2]['label_ids'])

{'input_ids': [0, 404, 5, 3749, 58, 13, 3034, 12, 19675, 12, 23414, 3749, 8, 963, 117, 6554, 50, 2257, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
def compute_accuracy(predictions, labels):
    correct = (predictions == labels).sum().item()
    total = labels.numel()
    accuracy = correct / total
    return accuracy

In [7]:
data_collator = DataCollatorForTokenClassification(tokenizer)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn = data_collator)
test_loader = DataLoader(test_ds, batch_size=32,collate_fn = data_collator)

In [8]:
from tqdm import tqdm
# Set up the optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 5

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label_ids = batch["label_ids"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=label_ids)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({"Loss": total_loss / len(progress_bar)})

    progress_bar.close()

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss/len(train_loader)}")

    # Evaluation
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            label_ids = batch["label_ids"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, predicted_labels = torch.max(outputs.logits, dim=-1)
            total_correct += (predicted_labels == label_ids).sum().item()
            total_samples += label_ids.numel()

    accuracy = total_correct / total_samples
    print(f"Accuracy: {accuracy}")
#Not to self: DO NOT RUN THIS CELL or RISK waiting 10 years

c:\users\eujin\appdata\local\programs\python\python39\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/5:   0%|                                                                                | 0/25 [00:00<?, ?it/s]You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Exception ignored in: <generator object tqdm.__iter__ at 0x000001C8A2394F90>
Traceback (most recent call last):
  File "c:\users\eujin\appdata\local\programs\python\python39\lib\site-packages\tqdm\std.py", line 1209, in __iter__
    self.n = n
KeyboardInterrupt: 


KeyboardInterrupt: 